In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv


# 문제 정의
10개의 아울렛 매장에서 1,500여 개의 제품에 대한 판매 데이터를 수잡했다. 예측 모델을 만들고 아울렛 특정 매장에서 각 제품의 판매금액을 예측하시오.
- 평가 기준은 RMSE로 평가
- label(target)은 판매금액(Item_Outlet_Sale)
- 제출 파일은 예측값만 result.csv 파일로 생성해 제출(컬럼명: pred, 1개)

# 데이터 불러오기

In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 탐색적 데이터 분석(EDA)

In [ ]:
# 크기 확인
train.shape, test.shape

((6818, 12), (1705, 11))

In [ ]:
# 데이터 샘플 확인
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1,1176.4686
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830


In [ ]:
# 데이터 자료형 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB


In [ ]:
# 기술통계량 확인
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,5656.000000,6818.000000,6818.000000,6818.000000,6818.000000
mean,12.872703,0.066121,140.419533,1997.885890,2190.941459
std,4.651034,0.051383,62.067861,8.339795,1706.131256
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026914,93.610050,1987.000000,836.577700
50%,12.600000,0.053799,142.448300,1999.000000,1806.648300
75%,17.000000,0.095273,185.060150,2004.000000,3115.944000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
# object컬럼 EDA
train.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDO19,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


In [ ]:
test.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,1705,1705,1705,1705,1235,1705,1705
unique,1077,5,16,10,3,3,4
top,DRD15,Low Fat,Fruits and Vegetables,OUT013,Medium,Tier 3,Supermarket Type1
freq,4,997,272,207,565,686,1103


In [ ]:
# 결측치 확인
train.isnull().sum()

,0
Item_Identifier,0
Item_Weight,1162
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,1940
Outlet_Location_Type,0


In [ ]:
test.isnull().sum()

,0
Item_Identifier,0
Item_Weight,301
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,470
Outlet_Location_Type,0


# 데이터 전처리

# 인코딩
object컬럼을 인코딩.
list로 컬럼을 만든다.

In [ ]:
#object 컬럼 선택
cols = list(train.columns[train.dtypes == object])
cols

['Item_Identifier',
 'Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [ ]:
# ID는 test와 train의 카테고리가 다르므로 제거하고 진행
cols.remove('Item_Identifier') #이때는 다시 저장안해도 바로 삭제 됨
print(cols)

['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [ ]:
# train test 병합후 레이블 인코딩
target = train.pop('Item_Outlet_Sales') # 타겟값 추출
print(train.shape, test.shape) # 칼럼 수 일치 확인.
df = pd.concat([train, test]) # 병합
print(df.shape)

(6818, 11) (1705, 11)
(8523, 11)


In [ ]:
#레이블 인코딩 진행
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cols:
  df[col] = le.fit_transform(df[col])

In [ ]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.500,1,0.006760,9,42.8112,1,1987,1,2,1
1,FDW11,12.600,1,0.048741,1,60.4194,1,1987,1,2,1
2,FDH32,12.800,1,0.075997,6,97.1410,1,1987,1,2,1
3,FDL52,6.635,2,0.046351,5,37.4506,2,2007,0,1,1
4,FDO09,13.500,2,0.125170,13,261.4910,1,1987,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...
1700,NCU18,NaN,1,0.097769,9,142.4496,4,1985,3,0,0
1701,FDY02,NaN,2,0.153457,4,264.0910,4,1985,3,0,0
1702,FDI09,20.750,2,0.129864,12,240.1880,3,2009,2,2,2
1703,DRH23,14.650,1,0.171282,7,54.0614,2,2007,0,1,1


In [ ]:
# 레이블 인코딩 다른방법
# for col in cols:
#   df[col] = df[col].astype('category').cat.codes


In [ ]:
# 다시 train과 test로 분할
train = df.iloc[:len(train)].copy()
test = df.iloc[len(test):].copy()
train.shape, test.shape

((6818, 11), (1705, 11))

In [ ]:
#결측치 채우기
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])

In [ ]:
#무의미한 ID제거
train.drop('Item_Identifier',axis=1,inplace=True)
test.drop('Item_Identifier',axis=1,inplace=True)
train.shape, test.shape

((6818, 10), (1705, 10))

In [ ]:
#검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5454, 10), (1364, 10), (5454,), (1364,))

# 머신러닝 학습 및 평가

1. 선형회귀(random_state 없음)

2. 랜덤 포레스트

3. LightGBM

In [ ]:
# 평가모형 import
from sklearn.metrics import mean_squared_error # MSE
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import r2_score # r2
from sklearn.metrics import root_mean_squared_error #RMSE

In [ ]:
# 선형 회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print('MSE:', result)

result = mean_absolute_error(y_val, y_pred)
print('MAE:', result)

result = r2_score(y_val, y_pred)
print('R2:', result)

result = root_mean_squared_error(y_val, y_pred)
print('RMSE:', result)

MSE: 1252760.7688254556
MAE: 858.0764527019547
R2: 0.5174353872927403
RMSE: 1119.267961135963


In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print('MSE:', result)

result = mean_absolute_error(y_val, y_pred)
print('MAE:', result)

result = r2_score(y_val, y_pred)
print('R2:', result)

result = root_mean_squared_error(y_val, y_pred)
print('RMSE:', result)

MSE: 1099573.400979785
MAE: 746.7442005703812
R2: 0.5764433037885593
RMSE: 1048.6054553452338


In [ ]:
#LightGBM
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1)

model.fit(X_train, y_train)
y_pred = model.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print('MSE:', result)

result = mean_absolute_error(y_val, y_pred)
print('MAE:', result)

result = r2_score(y_val, y_pred)
print('R2:', result)

result = root_mean_squared_error(y_val, y_pred)
print('RMSE:', result)



MSE: 1103759.0757334102
MAE: 735.2040013823017
R2: 0.5748309779824954
RMSE: 1050.599388793564


# 예측 및 결과 파일 생성

In [ ]:
pred = model.predict(test)
pred

array([1218.26901142,  849.02314896, 1861.74848167, ..., 3894.89741703,
        891.69774486, 1389.6366374 ])

In [ ]:
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv', index=False)

In [ ]:
pd.read_csv('result.csv')

,pred
0,1218.269011
1,849.023149
2,1861.748482
3,1587.280190
4,2803.891848
...,...
1700,280.316718
1701,629.940520
1702,3894.897417
1703,891.697745
